In [1]:
import pandas as pd
import numpy as np
import ast
import re
from zhon.hanzi import non_stops , stops
import os
import pickle
from opencc import OpenCC
# from transformers import RobertaForTokenClassification
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import RobertaTokenizer
from transformers import BertTokenizer , BertConfig , BertModel, XLNetTokenizer ,XLNetModel
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification,BertForPreTraining
from torch.autograd import Variable
from transformers import BertForPreTraining
from torch import nn
import json
import requests
pretrain_model_name = './chinese_roberta_wwm/'
my_pretrain = './pretrain_roberta_on_TBdata'

api_url = 'http://35.221.209.220/inference'

In [2]:
train_df = pd.read_csv("./tbrain/tbrain_train_split.csv")
train_df = pd.read_csv("./tbrain/tbrain_train.csv")
train_df = train_df.fillna('None')

test_df = pd.read_csv("./tbrain/tbrain_test.csv")

test_df = pd.read_csv("./tbrain/tbrain_test_split.csv")
test_df = test_df.fillna('None')

print(train_df.shape)
print(test_df.shape)


(4426, 4)
(526, 4)


In [3]:

def clean_string(content):
#     cc = OpenCC('t2s')
    content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
    content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
    content = re.sub(r"[%s]+" %stops, "。",content)
#     content = cc.convert(content)
    return content

def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para) 
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    return para.split("\n")


def combine_sentence(sentences):
    li = []
    string = ''
    for k in range(len(sentences)):
        sentence = sentences[k]
        if len(string) + len(sentence) < 510:
            string = string + sentence
        else:
#             原本是空的代表sentences太常
            if string == '':
                n = 510
                tmp_li = [sentence[i:i+n] for i in range(0, len(sentence), n)]
                string = tmp_li.pop(-1)
                li = li + tmp_li
            else:
                li.append(string)
                string = sentence
    if(string != ''):
        li.append(string)
    return li
        
    
    
    

In [4]:
tokenizer = XLNetTokenizer.from_pretrained('./chinese_xlnet_mid_pytorch/')

common_words = ['洗錢','貪污','政治','交保','違法']
max_seq_length = 512
train_x = []
train_input_ids = []
train_token_types = []
train_attention_mask = []
train_start_pos = []
train_end_pos = []
train_ckip_names = []
one_count = 0

ckip_names = train_df['ckip_names'].tolist()
train_name_ans = train_df['name'].tolist()
train_content = train_df['full_content'].tolist()




for i in range(len(train_content)):
    content = train_content[i]
    content = clean_string(content)
    content_ckip_names = ast.literal_eval(ckip_names[i])
    name_ans = train_name_ans[i]
    a_list = content_ckip_names
    split_content = cut_sent(content)
    chunks = combine_sentence(split_content)
    
    for chunk in chunks:
        for a in a_list:
            #     bert input
#                 input ids
            input_ids = tokenizer.encode(chunk, a)
            if(len(input_ids)>512):
                continue
#                 token type, 0 is question 1 is ans
            sep_index = input_ids.index(tokenizer.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(input_ids) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b
                
#                 attention mask 
            input_mask = [1] * len(input_ids)
                
            while len(input_ids) < max_seq_length:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)
                
            train_input_ids.append(input_ids)
            train_token_types.append(segment_ids)
            train_attention_mask.append(input_mask)
            train_x.append((chunk,a)) 
            train_ckip_names.append(content_ckip_names)
            
            
            start_pos = chunk.find(name_ans)
            start_pos_label = np.zeros(512)
            end_pos_label = np.zeros(512)
        
            if name_ans == 'None' or start_pos == -1:
                start_pos_label[0] = 1
                end_pos_label[0] = 1
                train_start_pos.append(start_pos_label)
                train_end_pos.append(end_pos_label)
            else:
                one_count += 1
                start_pos += 1
                end_pos = start_pos + len(name_ans)
                start_pos_label[start_pos] = 1
                end_pos_label[end_pos] = 1
                train_start_pos.append(start_pos_label)
                train_end_pos.append(end_pos_label)
            
       
        
    
train_input_ids = np.array(train_input_ids)
train_token_types  = np.array(train_token_types)
train_attention_mask = np.array(train_attention_mask)
train_start_pos_labels = np.array(train_start_pos)
train_end_pos_labels = np.array(train_end_pos)
train_ckip_names = np.array(train_ckip_names)

print(len(train_x))
print(train_input_ids.shape)
print(train_token_types.shape)
print(train_attention_mask.shape)
print(train_start_pos_labels.shape)
print(train_end_pos_labels.shape)
print(train_ckip_names.shape)




72201
(72201, 512)
(72201, 512)
(72201, 512)
(72201, 512)
(72201, 512)
(72201,)


In [35]:
tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)

common_words = ['洗錢','貪污','政治']
# common_words = ['洗錢']
max_seq_length = 512
CLS = '[CLS]'
SEP = '[SEP]'
test_x = []
test_input_ids = []
test_token_types = []
test_attention_mask = []
test_start_pos = []
test_end_pos = []
test_ckip_names = []

ckip_names = test_df['ckip_names'].tolist()
test_name_ans = test_df['name'].tolist()
test_content = test_df['full_content'].tolist()

one_count = 0
for i in range(len(test_content)):
    content = test_content[i]
    content = clean_string(content)
    content_ckip_names = ast.literal_eval(ckip_names[i])
    name_ans = test_name_ans[i]
    a_list = common_words + content_ckip_names
    split_content = cut_sent(content)
    chunks = combine_sentence(split_content)

    for chunk in chunks:
        for a in a_list:
            
            #     bert input
#                 input ids
            input_ids = tokenizer.encode(chunk, a)
            if(len(input_ids)>512):
                continue
#                 token type, 0 is question 1 is ans
            sep_index = input_ids.index(tokenizer.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(input_ids) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b
                
#                 attention mask 
            input_mask = [1] * len(input_ids)
                
            while len(input_ids) < max_seq_length:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            test_input_ids.append(input_ids)
            test_token_types.append(segment_ids)
            test_attention_mask.append(input_mask)
            test_x.append((chunk,a))
            test_ckip_names.append(content_ckip_names)

            
            
            start_pos = chunk.find(name_ans)
            start_pos_label = np.zeros(512)
            end_pos_label = np.zeros(512)
            if name_ans == 'None' or start_pos == -1:
                start_pos_label[0] = 1
                end_pos_label[0] = 1
                test_start_pos.append(start_pos_label)
                test_end_pos.append(end_pos_label)
            else:
                one_count += 1
                start_pos += 1
                end_pos = start_pos + len(name_ans)
                start_pos_label[start_pos] = 1
                end_pos_label[end_pos] = 1
                test_start_pos.append(start_pos_label)
                test_end_pos.append(end_pos_label)
            
       
        
    
test_input_ids = np.array(test_input_ids)
test_token_types  = np.array(test_token_types)
test_attention_mask = np.array(test_attention_mask)
test_start_pos_labels = np.array(test_start_pos)
test_end_pos_labels = np.array(test_end_pos)
test_ckip_names = np.array(test_ckip_names)
print(len(test_x))
print(test_input_ids.shape)
print(test_token_types.shape)
print(test_attention_mask.shape)
print(test_start_pos_labels.shape)
print(test_end_pos_labels.shape)
print(test_ckip_names.shape)




5471
(5471, 512)
(5471, 512)
(5471, 512)
(5471, 512)
(5471, 512)
(5471,)


In [4]:
class TrainDataset(Dataset):
    def __init__(self, 
                 input_ids,
                 token_types, 
                 attention_mask,
                 start_pos_label, 
                 end_pos_label):
        self.input_ids = input_ids
        self.token_type_ids = token_types
        self.attention_mask = attention_mask
        self.start_pos_labels = start_pos_label
        self.end_pos_labels = end_pos_label
    def __getitem__(self,idx):
        inputid = np.array(self.input_ids[idx])
        tokentype = np.array(self.token_type_ids[idx])
        attentionmask = np.array(self.attention_mask[idx])
        start_pos = self.start_pos_labels[idx]
        end_pos =  self.end_pos_labels[idx]
        return inputid , tokentype , attentionmask,  start_pos , end_pos
    
    def __len__(self):
        return len(self.input_ids)

class posClassfication_new(nn.Module):
    def __init__(self):
        super(posClassfication_new, self).__init__()
        self.start_task = nn.Sequential(
            nn.Linear(768, 1),

        )    
        self.end_task = nn.Sequential(
            nn.Linear(768, 1),
        ) 
#             
    def forward(self, start_x,end_x):
        start_x = start_x.double()
        end_x = end_x.double()
        
        start_out = self.start_task(start_x)
        end_out = self.end_task(end_x)
        
        return start_out,end_out
    

In [ ]:
# training


BATCH_SIZE = 1
# config = BertConfig.from_pretrained(my_pretrain,output_hidden_states=True)



trainset = TrainDataset(train_input_ids,
                        train_token_types,
                        train_attention_mask,
                        train_start_pos_labels,
                        train_end_pos_labels)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = XLNetModel.from_pretrained('./chinese_xlnet_mid_pytorch/')
model = model.to(device)
model.eval()


pos_model = posClassfication_new()
pos_model = pos_model.to(device)
pos_model = pos_model.double()
pos_model.train()

pos_loss = nn.CrossEntropyLoss()

EPOCHS = 5
optimizer = torch.optim.Adam(pos_model.parameters(), lr=3e-5)

for epoch in range(EPOCHS):
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors ,  segments_tensors , masks_tensors, start_pos_labels , end_pos_labels = [t.to(device) for t in data]
        
        
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        

        bert_all_768 = bert_outputs[0]
        
        mini_batch = bert_all_768.size()[0]
        optimizer.zero_grad()
        start_pred , end_pred = pos_model(bert_all_768,bert_all_768)
        
        start_pred = start_pred.reshape((mini_batch,512))
        end_pred = end_pred.reshape((mini_batch,512))


        
        start_pos_labels = torch.argmax(start_pos_labels,dim=1)
        end_pos_labels = torch.argmax(end_pos_labels,dim=1)

        start_loss = pos_loss(start_pred,start_pos_labels)
        end_loss = pos_loss(end_pred,end_pos_labels)
        
        
        loss = (start_loss + end_loss) / 2
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    torch.save(pos_model,'./TB_multispan/XLNet_QAdata' + str(epoch) + '.pkl')
    print('[epoch %d] loss: %.3f' %(epoch + 1, running_loss))
        

        
# with open('./TB_multispan/train_all_768_x.pickle', 'wb') as handle:
#     pickle.dump(all_768_x, handle, protocol=pickle.HIGHEST_PROTOCOL) 


device: cuda:0


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type posClassfication_new. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[epoch 1] loss: 52550.689
[epoch 2] loss: 22766.841
[epoch 3] loss: 17538.158
[epoch 4] loss: 14828.861


In [5]:
def eval(pred, ans):
    if bool(pred) is not bool(ans):
        return 0
    elif not pred and not ans:
        return 1
    else:
        pred = set(pred)
        ans = set(ans)
        interaction_len = len(pred & ans)
        if interaction_len == 0:
            return 0

        pred_len = len(pred)
        ans_len = len(ans)
        return 2 / (pred_len / interaction_len + ans_len / interaction_len)


def eval_all(pred_list, ans_list):
    assert len(pred_list) == len(ans_list)
    return sum(eval(p, a) for p, a in zip(pred_list, ans_list)) / len(pred_list)

In [25]:
def get_ans(test_name):
    config = BertConfig.from_pretrained('./pretrain_roberta_on_TBdata/'  + 'config.json',output_hidden_states=True)
    tokenizer = BertTokenizer.from_pretrained('./pretrain_roberta_on_TBdata/')

    test_name = test_name
    test_input_dict = tokenizer.encode_plus(test_name, 
                                         add_special_tokens=True,
                                         max_length=5,
                                            truncation=True,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')

#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     print("device:", device)

    model = BertModel.from_pretrained('./pretrain_roberta_on_TBdata/',config=config)
    model = model.to(device)
    model.output_hidden_states = True
    model.eval()
    name_model.eval()


    bert_outputs = model(input_ids = test_input_dict['input_ids'].to(device), 
                        token_type_ids = test_input_dict['token_type_ids'].to(device), 
                        attention_mask= test_input_dict['attention_mask'].to(device))  

    pool_cls = bert_outputs[1]
    pool_cls = pool_cls.double()


    logits = name_model(pool_cls)
    
    #is_name =torch.nn.Softmax(logits)
    is_name = torch.argmax(logits)

    return(is_name)


In [28]:
BATCH_SIZE = 1


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
tokenizer = XLNetTokenizer.from_pretrained('./chinese_xlnet_mid_pytorch/')
model = XLNetModel.from_pretrained('./chinese_xlnet_mid_pytorch/')
model = model.to(device)
model.eval()
model.eval()
# pos_model.eval()

max_seq_length = 512
class TestDataset(Dataset):
    def __init__(self,input_ids ,token_type_ids , attention_mask , text):
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.text = text

    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        text = self.text[idx]

        return inputid , tokentype , attentionmask, text
    
    def __len__(self):
        return len(self.input_ids)
    
class NameModel(nn.Module):
    def __init__(self):
        super(NameModel, self).__init__()
        self.name_task = nn.Sequential(
            nn.Linear(768, 768),
            nn.Tanh(),
            nn.Linear(768,2)
        )    
         
    def forward(self, x):
        x = x.double()
        out = self.name_task(x)
        return out


pos_model = torch.load('./TB_multispan/XLNet_QAdata3.pkl')
pos_model = pos_model.to(device)
pos_model.eval()

all_scroe = 0
test_df = pd.read_csv("./tbrain/tbrain_test.csv")
test_df = test_df.fillna('None')
names =  test_df['name'].tolist()
contents = np.array(test_df['full_content'].tolist())
ckip_names = test_df['ckip_names'].tolist()


name_model = torch.load('TB_multispan/name_model.pkl') 
name_model = name_model.to(device)
name_model.eval()


x = []
binary_y = []
names_y  = []
content_all_names = []


valid_count = 0
score = 0.0


for i in range(len(contents)):
    content = contents[i]
    content = clean_string(content)
    content_ckip_names = ast.literal_eval(ckip_names[i])

    if(content=='nan'):
        continue

    name_li  = ast.literal_eval(names[i])
    my_pred_name_list = []
    
    if len(name_li) == 0:
        score += eval(my_pred_name_list,name_li)
        valid_count += 1
        continue
        
    

    split_content = cut_sent(content)
    chunks = combine_sentence(split_content)
    
    
#     a_list = content_ckip_names
    a_list = ['洗錢','貪污','交保']
    
    test_input_ids = []
    test_attention_mask = []
    test_token_types = []
    test_x = []
    
    for chunk in chunks:
        for a in a_list:
            input_ids = tokenizer.encode(chunk, a)
            if(len(input_ids)>512):
                continue
#                 token type, 0 is question 1 is ans
            sep_index = input_ids.index(tokenizer.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(input_ids) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b
                
#                 attention mask 
            input_mask = [1] * len(input_ids)
                
            while len(input_ids) < max_seq_length:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)
                
            test_input_ids.append(input_ids)
            test_token_types.append(segment_ids)
            test_attention_mask.append(input_mask)
            test_x.append(chunk)
            
            
    test_input_ids = np.array(test_input_ids)
    test_token_types = np.array(test_token_types)
    test_attention_mask = np.array(test_attention_mask)
            
    

    testset = TestDataset(test_input_ids, test_token_types , test_attention_mask, test_x)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)

    my_pred_name_list = []
    for data in testloader:

        tokens_tensors ,  segments_tensors , masks_tensors = [t.to(device) for t in data[:-1]]

        text = data[-1]

        bert_outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)

        bert_all_768 = bert_outputs[0]
        

        mini_batch = bert_all_768.size()[0]
        bert_all_768 = bert_all_768.double()
        
#         pool_cls = bert_outputs[1]
#         pool_cls = pool_cls.double()

        start_pred , end_pred = pos_model(bert_all_768, bert_all_768)

#         binary_pred = torch.argmax(binary_pred,dim=-1)
            
        start_pred = start_pred.reshape((mini_batch,512))
        end_pred = end_pred.reshape((mini_batch,512))


        topk = 5
        myrange = 20
        start_topk_indices = torch.topk(start_pred, topk).indices
        end_top_k_indices = torch.topk(end_pred,topk).indices

        all_indices = torch.cat([start_topk_indices,end_top_k_indices] , dim=-1)
        all_indices = torch.unique(all_indices)

        for i in range(all_indices.size()[0]):
            start_index = all_indices[i]
            start_index -= 1
#             print('start_index: ' ,start_index)

            ans_string = ''

            if (start_index + myrange < 512) and (start_index - myrange >= 0):
                ans_string = text[0][start_index - myrange:start_index + myrange]
            elif start_index + myrange > 512:
                ans_string = text[0][start_index-myrange:-1]
            elif start_index - myrange <= 0 :
                ans_string = text[0][0:start_index+myrange]
            else:
                print('out of range ', start_index)

            for ckip_name in content_ckip_names:
                if(ckip_name in ans_string and len(ckip_name) >= 2):
                    if ckip_name not in my_pred_name_list:
                        if get_ans(ckip_name).item() > 0:
                            my_pred_name_list.append(ckip_name)
        

    
    print('----------------')
    print('pred : ' ,my_pred_name_list)
    print('ans : ' ,name_li)
        
    score += eval(my_pred_name_list,name_li)
    valid_count += 1
#     break
        

print(score/valid_count)
print(valid_count)


device: cuda:0
----------------
pred :  ['李訓成', '王宇正', '蔡開宇']
ans :  ['蔡開宇', '王宇正', '李訓成']
----------------
pred :  ['張永泉', '郭明賓', '蔡英俊']
ans :  ['張永泉', '郭明賓']
----------------
pred :  ['李瑞廷']
ans :  ['李瑞廷']
----------------
pred :  ['牟孝儀', '陳學敏', '牟明哲']
ans :  ['陳學敏', '牟孝儀']
----------------
pred :  ['許祈文']
ans :  ['許祈文']
----------------
pred :  ['黃顯雄', '黃世陽', '錢利忠', '曹興誠', '宣明智']
ans :  ['黃世陽', '黃顯雄']
----------------
pred :  ['呂建安', '呂東英', '林裕豐', '劉昌松']
ans :  ['黃淑頻', '呂建安']
----------------
pred :  ['蕭博文', '章民強', '章啟光', '章啟明', '李恆隆', '賴永吉']
ans :  ['章民強', '章啟光', '章啟明']
----------------
pred :  ['吳昇儒', '陳正達', '黃志豪']
ans :  ['張建生', '林宏彬', '張宜豐', '陳正達', '黃志豪']
----------------
pred :  []
ans :  ['傅春生']
----------------
pred :  ['陳慰慈']
ans :  ['莊錫根']
----------------
pred :  ['朱國榮', '劉慶珠']
ans :  ['朱國榮', '劉慶珠']
----------------
pred :  ['張志偉', '周麗真', '陳逢璿']
ans :  ['周麗真', '張志偉', '陳逢璿']
----------------
pred :  ['孫幼英', '陳慰慈', '陳柏文']
ans :  ['江國貴', '孫幼英', '林偉強', '蘇芸樂', '鍾素娥', '張玉鳳']
---

In [34]:
# testing topk 


BATCH_SIZE = 1
config = BertConfig.from_pretrained(my_pretrain,output_hidden_states=True)



class TestDataset(Dataset):
    def __init__(self, 
                 input_ids,
                 token_types, 
                 attention_mask,
                 start_pos_label, 
                 end_pos_label, text):
        self.input_ids = input_ids
        self.token_type_ids = token_types
        self.attention_mask = attention_mask
        self.start_pos_labels = start_pos_label
        self.end_pos_labels = end_pos_label
        self.text = text
    def __getitem__(self,idx):
        inputid = np.array(self.input_ids[idx])
        tokentype = np.array(self.token_type_ids[idx])
        attentionmask = np.array(self.attention_mask[idx])
        start_pos = self.start_pos_labels[idx]
        end_pos =  self.end_pos_labels[idx]
        text = self.text[idx]
        return inputid , tokentype , attentionmask,  start_pos , end_pos , text
    
    def __len__(self):
        return len(self.input_ids)

testset = TestDataset(test_input_ids,
                        test_token_types,
                        test_attention_mask,
                        test_start_pos_labels,
                        test_end_pos_labels,
                       test_x)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = BertModel.from_pretrained('./pretrain_roberta_on_TBdata/',config=config)
model = model.to(device)
model.output_hidden_states = True
model.eval()


pos_model = torch.load('./TB_multispan/pos_model_single_task_QAdata.pkl')
pos_model.eval()
pos_loss = nn.CrossEntropyLoss()


with torch.no_grad():
    for data in testloader:
        
        tokens_tensors ,  segments_tensors , masks_tensors, start_pos_labels , end_pos_labels = [t.to(device) for t in data[:5]]
        
        text = data[-1]
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        

        bert_all_768 = bert_outputs[0]
        
        mini_batch = bert_all_768.size()[0]
        optimizer.zero_grad()
        start_pred , end_pred = pos_model(bert_all_768,bert_all_768)
        
        start_pred = start_pred.reshape((mini_batch,512))
        end_pred = end_pred.reshape((mini_batch,512))


#         start_pred = nn.functional.log_softmax(start_pred,dim=1)
#         end_pred = nn.functional.log_softmax(end_pred,dim=1)
        
#         start_pos_labels = torch.softmax(start_pos_labels,dim=1)
#         end_pos_labels = torch.softmax(end_pos_labels,dim=1)
        
        start_pos_labels = torch.argmax(start_pos_labels,dim=1)
        end_pos_labels = torch.argmax(end_pos_labels,dim=1)
        
        
        if(start_pos_labels[0] > 0):
            print('===========================')
            print(text[0])
            print('ans is: ', text[0][0][start_pos_labels[0]-1:end_pos_labels[0]-1])
            print('===========================')
        else:
            continue
            
        pred_mat = start_pred * end_pred.t()
        H, W = pred_mat.shape
        pred_mat = pred_mat.view(-1)
        topk = 3
        indices = torch.topk(pred_mat, topk).indices
        two_d_indices = torch.cat(((indices // W).unsqueeze(1), (indices % W).unsqueeze(1)), dim=1)

        for pos in two_d_indices:
            start_index = pos[1]-1
            end_index = pos[0]-1
            print(start_index,end_index)
            try:
                if(start_index < end_index):
                    print('model output ans ' + str(i) + ' ' +  text[0][0][start_index:end_index])
                elif(start_index == end_index):
                    print('model output ans ' + str(i) + ' ' +  text[0][0][start_index])
                else:
                    print('model output ans ' + str(i) + ' ' +  text[0][0][end_index:start_index])
                    
            except:
                print('out of range')
        
        
        




device: cuda:0
('蘋果西打母公司掏空案總經理聲押禁見董座15萬交保大飲總經理孫幼英昨深夜移送新北檢複訊訊後遭聲押禁見記者陳慰慈攝2019-04-1206:00:12記者陳慰慈新北報導生產知名飲料蘋果西打的大西洋飲料公司1213因財報難產爆出掏空疑雲目前股票已暫停交易檢調昨依違反證券交易法非常規交易特別背信罪等罪約談大飲董事長江國貴登記負責人總經理孫幼英實際負責人獨立董事林偉強蘇芸樂財務經理鍾素娥張玉鳳等6人到案蘇芸樂昨晚訊後以3萬5千元交保今凌晨2時許依涉犯重罪有串證逃亡之虞將孫幼英聲押禁見江國貴15萬元林偉強10萬元鍾素娥40萬元張玉鳳25萬元交保大飲公司近日深陷財務經營風暴去年第3季與關係人簽約以64億元取得位於台南佳里高雄湖內及新北市新店等3筆不動產而與大飲交易的國信旭順公司董事名單高度重疊大飲總經理孫幼英身兼國信董事且國信又是大飲旭順的監察人旭順的監察人則是國信公司治理惹議請繼續往下閱讀此外國信旭順公司登記地址也是大飲位於新北市三重區的辦公室等於都是大飲能實質掌控的公司讓外界有左手賣右手的掏空質疑而林偉強蘇芸樂分別擔任國信食品代協理國信食品課長卻又是大飲獨立董事不符金管會規定讓會計師對大飲去年財',)
ans is:  林偉強
tensor(491, device='cuda:0') tensor(491, device='cuda:0')
model output ans 2 會
tensor(489, device='cuda:0') tensor(491, device='cuda:0')
model output ans 2 定讓
tensor(491, device='cuda:0') tensor(489, device='cuda:0')
model output ans 2 定讓
('參選嘉義市東區議員失利的藍力凱及梅山鄉民代表當選人許文豪因競選期間涉嫌賄選嘉義地檢署檢察官22日依選罷法第99條第1項對2人提起公訴並針對許文豪提出當選無效之訴2人目前仍在羈押中今年10月間檢調接獲情資指藍力凱的李姓樁腳有買票行為大動作搜索李姓樁腳住處藍向檢察官表示與李姓樁腳熟識但並無交付他行賄李姓樁腳卻坦承曾與藍商議找陳姓樁腳幫忙拉票藍於是準備裝有20萬元的茶葉罐交付予李再將款項轉交給陳多名樁腳指證歷歷加上藍曾刪除與李姓樁腳的通話紀錄刻意切割行

('金錢豹餐飲集團等擔任工程部副理的洪勝明負責寶麗金餐飲集團下的聚品軒婚宴會館寶麗金鑫悅富餐廳-崇德店裝修工程涉嫌向多家承攬廠商索取5至15回扣共收取回扣1318萬8000元經老闆袁昶平發現向調查局台中市調查處提出檢舉台中地檢署今天依背信罪將洪勝明起訴洪勝明否認背信犯行辯稱他是鼎盛冷氣行的技術師不清楚鼎盛冷氣行是否為告訴人首元公司的關係企業他沒有太大權力任何事情都是范姓工程處處長所決定他向廠商所收取的不是回扣而是工程介紹費且未曾對廠商施壓或刁難洪的律師也辯護稱得標施作的廠商僅是將利潤撥付一些給洪感謝洪介紹工作機會起訴書指出洪勝明52歲於2004年間開始受雇於袁昶平所經營的金錢豹餐飲機構於袁昶平在2015年設立首元投資開發股份有限公司後仍繼續在該公司旗下的藏格教育集團金錢豹娛樂集團及寶麗金餐飲集團等事業體內擔任工程部副理2016年2月間洪勝明經袁昶平指示與不知情的范姓工程處處長負責寶麗金集團下的聚品軒婚宴會館整修工程時以收取工程介紹費向承攬廠商收取工程款5至15的回扣損害首元公司的財產及利益同年間洪勝明向施作聚品軒婚宴會館木作裝修工程的慧友室友設計公司黃姓業者索取回扣106萬向施作天花板木工',)
ans is:  洪勝明
tensor(413, device='cuda:0') tensor(413, device='cuda:0')
model output ans 2 收
tensor(413, device='cuda:0') tensor(340, device='cuda:0')
model output ans 2 集團及寶麗金餐飲集團等事業體內擔任工程部副理2016年2月間洪勝明經袁昶平指示與不知情的范姓工程處處長負責寶麗金集團下的聚品軒婚宴會館整修工程時以
tensor(413, device='cuda:0') tensor(487, device='cuda:0')
model output ans 2 收取工程介紹費向承攬廠商收取工程款5至15的回扣損害首元公司的財產及利益同年間洪勝明向施作聚品軒婚宴會館木作裝修工程的慧友室友設計公司黃姓業者索取回
('工程的久德裝潢公司施姓老闆索取回扣198萬餘元向施作拆除及泥作工程的欣岳工程行趙姓業者索取回扣267萬元此外洪勝明向施作餐廳雜項鐵工的勝霖金屬公司鄒姓業者索取回扣110萬6000向施作水電工程的

('國民黨前新竹市議員李黃錦燕更二審判刑4年資料照記者楊國文台北報導國民黨籍前新竹市議員李黃錦燕被控從2000年起利用人頭虛報議會補助的助理費近10年來合計領走932萬元一審依貪污治罪條例的利用職務詐取財務罪重判她10年且932萬元應予追繳發還市議會上訴後高等法院更一審曾以議會沒有審查公費助理工作內容的權限以罪證不足改判無罪不過高院更二審今上午又以李黃錦燕貪污罪行明確判她4年徒刑褫奪公權3年還可上訴李黃錦燕今上午未出庭聆判無法得知她對判決結果的看法法界指出因更一審曾判李黃錦燕無罪今更二審改判4年徒刑李黃錦燕上訴的可能性高請繼續往下閱讀李黃錦燕曾任新竹市第三到七屆議員任期到2010年2月8日為止後來競選連任失利檢警調查李黃錦燕從2000年起陸續向議會申報9人領取每月4萬元的助理費其中有些是李黃錦燕先生李榮勝診所的護士有些則是在外當保母縫紉工連她的兒子也名列虛報人頭之一而實際聘請的三名助理每月只領2萬元現金勞健保還須自付新竹地院審理時李黃錦燕指稱10年來所聘用的人都實際有擔任助理工作助理費都是用現金給他們有些助理認為錢給得太多希望她把一部分拿去做公益或是提供服務處開銷所有的錢都是助理同意奉獻的',)
ans is:  李黃錦燕
tensor(491, device='cuda:0') tensor(495, device='cuda:0')
model output ans 2 都是助理
tensor(495, device='cuda:0') tensor(495, device='cuda:0')
model output ans 2 同
tensor(493, device='cuda:0') tensor(495, device='cuda:0')
model output ans 2 助理
('她付出的公益費用遠大於請領的費用不過法官認為李黃錦燕長期擔任民代竟以人頭助理或短報助理薪資方式詐領議會的公費助理薪資犯後飾詞狡辯矢口否認犯罪依貪污治罪條例中利用職務詐取財物罪重刑10年上訴後高等法院改判8年徒刑褫奪公權6年全案更一審時大逆轉改判李黃錦燕無罪檢方不服提起上訴經最高法院撤銷發回更二審今改判4年徒刑褫奪公權3年不用抽不用搶現在用APP看新聞保證天天中獎',)
ans is:  李黃錦燕
tensor(397, device='cuda:0') tensor(4

('記者黃建華高雄報導高雄一名廖姓男子1993年涉嫌和人以漁船走私多達66塊海洛英磚入境藏在小港區大林蒲的魚塭準備慢慢出貨卻被警方查獲廖男落網後棄保潛逃迄今未到案由於司法追訴時效僅到去年8月27日廖男確定逃過法律制裁檢方公訴指控被告廖聰昇與王春容及綽號阿楚真實姓名年籍均不詳之香港男子共同意圖營利基於私運管制物品進入台灣及販賣毒品海洛因之犯意聯絡先由王春容於1993年2月初經蔡登裕介紹以新台幣200萬元僱用郭再旺以其漁船接運毒品後被告即於同年3月30日下午6時50分會同王春容及阿楚前往中國汕頭洽購海洛因並於同年4月1日上午8時許在廣東省南澳港將購得之海洛因磚交由綽號發仔真實姓名年籍均不詳之中國男子以木船載運至南澳港外某不知名小島再轉交大順號漁船船長即郭再旺船員郭義禮及吳勝夫以該漁船私運回台於同年4月5日上午行駛至雲林縣沿海後交由蔡登裕以舢舨接駁上岸復交由綽號小鹿阿東之人於同日下午4時許載往被告廖聰昇與王春容共有之高雄市小港區大林埔漁塭藏放並分由王協模小鹿販賣予綽號阿三及不特定之人施用警方於同年4月7日上午11時35分許在王協模所有之自小客車引擎上查獲海洛因140公克復於同日中午在前開漁塭查獲',)
ans is:  王協模
tensor(488, device='cuda:0') tensor(490, device='cuda:0')
model output ans 2 於同
tensor(490, device='cuda:0') tensor(490, device='cuda:0')
model output ans 2 日
tensor(486, device='cuda:0') tensor(490, device='cuda:0')
model output ans 2 克復於同
('捲帶式封裝ChiponFilm簡稱COF覆晶載板tape領先廠商頎邦科技所擁有的蝕刻技術等遭受前總經理李宛霞副總經理黃梅雪及離職員工侵害案件高雄地檢察署依違反營業祕密法偵查起訴檢方起訴指出己由頎邦科技合併的欣寶公司在102年間自行研發的最新蝕刻製程技術及欣寶公司向日本技術移轉所取得的蝕刻技術等營業秘密遭時任欣寶公司的李宛霞總經理黃梅雪副總經理及關鍵技術團隊包括陳姓部經理等人利用職務上的權限及機會以電子檔案方式複製侵占資料量極大該等人員於離職後前往頎邦科技的競爭對手易華公司

('寶德資訊負責人曾昭榮以圈購誠品書店台灣大車隊等未上市櫃股票為利誘僱用業務員騙過上千名被害人投資非法121億元曾昭榮偵查中逃亡遭通緝法院依違反銀行法將姚柏丞等10名共犯判刑另查出知名股市炒手鄒勝的胞妹鄒春香等7人涉案依職權告發台北地檢署今追加起訴曾昭榮早年即假藉通報股市明牌以美女業務員吸金兩度被法辦後來另起爐灶對外宣稱可向承銷商提前圈購誠品書店台灣大車隊股票檢調再辦吸金案命令凍結有關帳戶扣得犯罪集團贓款4億4621萬元在陳効亮家中查扣現金1100萬元在曾昭榮家中查扣現金60萬元判決指出曾男在2011年7月間以寶德資訊公司曾昭榮姚柏丞雙盈行銷實業公司等名義宣稱可精準通報股市名牌招攬投資人入會吸金業務陳効亮等人對外招攬不特定投資人圈購未上市櫃股票與客戶約定投資報酬為3個月可回收8到10獲利並歸還本金實際報酬比例及年利率約在40到60詐騙投資客出錢3年間總計吸金118億多元被害人高達3000人次曾男被逮後獲100萬元交保但他隨即棄保逃亡台北地院將負責幫他招攬業務的被告陳効亮判刑12年梁柱11年9月姚柏丞11年6月馮一塵11年蘇雅玲9年林夢珍8年杜嘉珊5年劉人鳳4年6月等案經上訴高院認為其中4名',)
ans is:  陳効亮
tensor(347, device='cuda:0') tensor(347, device='cuda:0')
model output ans 2 歸
tensor(327, device='cuda:0') tensor(347, device='cuda:0')
model output ans 2 約定投資報酬為3個月可回收8到10獲利並
tensor(332, device='cuda:0') tensor(347, device='cuda:0')
model output ans 2 酬為3個月可回收8到10獲利並
('香港海關本周三23日)於機場入境大堂例行檢查2名從秘魯利馬出發經法國巴黎轉機抵港的男女時在2人的行李篋內共發現10支以泡泡紙及膠紙包裹的紅酒初步調查後證實檢獲的10支液態可卡因共重約22公斤市值約2300萬港元2名被控販毒罪的男女今早26日)被解往西九龍裁判法院提堂暫時毋須答辯控方要求將案押後至今年12月23日再提堂期間對涉案懷疑毒品作化驗2人續還押監房看管本案兩名被告分別是28歲女子沈陳婕楹及39歲男子沈駿州均持

...

('玩遊戲賺大錢勝威集團非法吸金3人羈押勝威國際集團官網顯示此集團總裁右與副總裁左分別是AlanLim與拿都MynusLee記者張瑞楨翻攝2019-04-1720:24:05記者張瑞楨台中報導總部設於柬埔寨金邊已被中國馬來西亞等國家掃蕩或列為黑名單的勝威國際集團其位於台灣的據點前天被調查局搜索一併傳喚主嫌李承孺與共犯證人共22人台中地檢署今天依涉嫌觸犯銀行法聲請羈押主嫌李承孺幹部詹國霆黃清政等3人獲准其餘共犯交保或釋回這個集團號稱在世界各地有逾12萬名會員卻以投資賭場玩遊戲賺大錢等說詞非法吸金投資100美金3年後獲利可達37倍吸引民眾爭相投資粗估在台灣已非法吸金數億元勝威國際集團的網站顯示該集團是由AlanLim與拿督MynusLee於2015年創立兩人擔任總裁與副總裁總部設立於柬埔寨首都金邊旗下子集團跨足金融賭博娛樂貸款學校教育與慈善團體請繼續往下閱讀該集團強調他們設計出一套前所未有的營銷方式及商業回饋方案結合權威娛樂金融機構迅速在全球多個地區發展讓客戶在彈指間就能獲得優質的投資及理財機會不過這個集團在中國非法吸金達2000萬元人民幣台幣1億餘元去年被中國官方掃蕩馬來西亞也禁止該集團運作',)
ans is:  黃清政
tensor(468, device='cuda:0') tensor(468, device='cuda:0')
model output ans 2 萬
tensor(468, device='cuda:0') tensor(371, device='cuda:0')
model output ans 2 團體請繼續往下閱讀該集團強調他們設計出一套前所未有的營銷方式及商業回饋方案結合權威娛樂金融機構迅速在全球多個地區發展讓客戶在彈指間就能獲得優質的投資及理財機會不過這個集團在中國非法吸金達2000
tensor(470, device='cuda:0') tensor(468, device='cuda:0')
model output ans 2 萬元
('亦即俗稱的列入黑名單檢調調查此集團於2016年起由主嫌李承孺邀集詹國霆黃清政等共犯在台灣設置據點吸金打出理財遊戲輕鬆玩財富自由跟著來在家輕鬆賺等口號去年1月還舉辦SVI勝威國際二周年年會的柬埔寨考察活動他們宣稱該集團橫跨柬埔寨馬來西亞與中國三個國家投資賭場等事業於中國打造4億美元的酒店在柬

('纏訟13年分時度假吸金13億英籍主謀更審判刑6年2006年時以分時度假為幌子的吸金詐騙集團吸金6年共詐得1億2942萬元經過纏訟13年高等法院更一審今依違反銀行法非法經營銀行業務罪對相關人等判刑資料照2019-05-3117:49:58記者張文川台北報導刑事警察局2006年破獲以分時度假為幌子的吸金詐騙集團吸金6年共詐得1億2942萬元經過纏訟13年高等法院更一審今依違反銀行法非法經營銀行業務罪加重改判英國籍主謀倪菲爾6年有期徒刑凌鳳琴等4名經理各判刑3年8月至4年半全案仍可上訴由於案件已審理逾8年適用速審法減刑規定因此倪菲爾等5人皆獲減刑但刑度仍比一審重台灣共犯劉穎谷陳志明各判刑4年6月凌鳳琴李泗源各判3年8月另一經理曾杭皆依常業詐欺罪判刑10月適用96年減刑條例減為5月6人犯罪不法所得一併宣告沒收請繼續往下閱讀判決指出倪菲爾等人於2001年2月起以樂吉美台灣分公司名義對外吸收CVC國際旅遊交換服務會員誘騙會員繳交2萬至70萬元會費宣稱會員可透過交換服務使用全球超過5500家渡假村並享受低廉的旅遊服務實則只是幌子集團還推出5年現金回饋計畫向會員宣稱5年後可以拿回本金或更高的金額實際上',)
ans is:  劉穎谷
tensor(421, device='cuda:0') tensor(479, device='cuda:0')
model output ans 2 會費宣稱會員可透過交換服務使用全球超過5500家渡假村並享受低廉的旅遊服務實則只是幌子集團還推出5年現金回饋計畫向會
tensor(421, device='cuda:0') tensor(476, device='cuda:0')
model output ans 2 會費宣稱會員可透過交換服務使用全球超過5500家渡假村並享受低廉的旅遊服務實則只是幌子集團還推出5年現金回饋計
tensor(421, device='cuda:0') tensor(442, device='cuda:0')
model output ans 2 會費宣稱會員可透過交換服務使用全球超過55
('卻無法使會員領回會費使會員陷入錯誤而簽約付費集團2006年4月又推出新點子想再剝會員一層皮向會員推銷VIPASIA會員卡佯稱只須再繳交差額就可將先前繳納的CVC會費或其他公司的會籍款項連同這次的會費合併計算總金額後分割成每

('記者黃捷台北報導毒販林輝宏1993年涉嫌以免費旅遊泰國為餌招攬6名男子吞食海洛因毒球方式從泰國走私毒品入台是國內首起吞毒球走私案後有2人因毒球在體內破裂毒發身亡林男事後遭控走私運毒及殺人罪但他逃亡已久運毒部分逾通緝時效台北地院判免訴殺人罪部分則因家屬聲請死亡宣告獲准台北地檢署認定林已死亡今處分不起訴檢方調查林輝宏與共犯魯德海1993年間找來張建華吳東明吳東明楊自立呂宗南裴振福劉吉雄以吞海洛因毒球走私為條件招待6人免費遊泰國若成功將毒品運回台灣可再另賺10萬元林輝宏先至泰國買毒並分裝成每顆5公克的毒球魯德海則協助代辦6男的出境手續6男抵達泰國後與林碰頭10月7日先讓呂宗南劉裴振福3人各吞下40多顆毒球後隔天搭機返台成功走私毒品同月11日林輝宏再將毒球交給張吳楊張吞20餘顆吳吞30餘顆楊則吞12顆但吳男吞下不久毒球就破裂當場毒發身亡張楊隔日搭機返台2人也因毒球破裂在飛機上身體不適2人抵台後在機場分手楊男自覺撐不住立刻請友人載他到醫院急診陸續排出11顆毒球撿回一命張男最後則中毒死亡檢警獲報後循線逮獲主嫌林輝宏魯德海及倖存的另4名運毒者北檢依懲治走私條例肅清煙毒條例起訴6人吳張則因死亡獲不起訴',)
ans is:  劉吉雄
tensor(506, device='cuda:0') tensor(492, device='cuda:0')
model output ans 2 則因死亡獲不起訴
tensor(488, device='cuda:0') tensor(492, device='cuda:0')
model output ans 2 6人吳張
tensor(492, device='cuda:0') tensor(492, device='cuda:0')
model output ans 2 則
('前網虎公司創辦人跨越科技董事長李奇申為取得陸客來台通關查驗系統兩項標案涉嫌以餽贈股票現金投資乾股代繳房貸等四大行賄手法買通內政部資訊中心主任移民署前資訊組長施明德護航得標台北地檢署昨天兵分廿一路搜索施李等人住居所帶回十三名官商人頭漏夜偵訊全案依違背職務行收賄洗錢罪偵辦除李奇申施明德被偵訊外廉政署也同步約談施明德妻子翁美惠姊姊張施麗滿姊夫張慶宗大嫂施吳素貞岳母賴月菊李奇申女友黃詩涵到案內政部表示部長徐國勇上任後發覺施的行為有異已免去他管理採購業務權限及擔任各項評

('利誘街友當人頭1張芭樂票跳票98億男子林秉勳及盧文凱涉嫌販售芭樂票近3年跳票81億餘元資料照2019-08-1605:30:00記者陳慰慈新北報導男子林秉勳原名林文良及盧文凱涉嫌販售芭樂票近3年跳票81億餘元其中盧嫌一張支票跳票金額高達98億元致我國跳票金額暴增影響台灣在亞太防制洗錢組織APG洗錢防制評鑑結果新北地檢署昨依詐欺取財等罪嫌起訴林男盧男及共犯等7人林秉勳以街友當人頭開立公司販售俗稱芭樂票的空頭支票跳票4750萬餘元2012年被判刑2年4個月確定林男出獄後又重操舊業請繼續往下閱讀3年跳票81億7嫌起訴起訴指出2015年起林男以自己名義或情商不知情子女擔任公司負責人另以1萬5千元至2萬元不等代價利誘何姓馬姓謝姓譚姓街友與失業者擔任資本額最高50萬元的公司負責人並向銀行申請支票簿以每張支票2800元賣給芭樂票集團盤商並命楊男前往交付芭樂票林男則預知芭樂票集團成員特定的退票時間並依客戶需求開立不同票期長短的芭樂票2016年7月至去年4月林男共賣出3172張芭樂票獲利888萬餘元跳票50億餘元而男子盧文凱也透過仲介向林男找來當公司登記負責人的何姓街友購買15張芭樂票並開出5張超過億元',)
ans is:  林秉勳
tensor(506, device='cuda:0') tensor(472, device='cuda:0')
model output ans 2 司登記負責人的何姓街友購買15張芭樂票並開出5張超過億元
tensor(506, device='cuda:0') tensor(481, device='cuda:0')
model output ans 2 街友購買15張芭樂票並開出5張超過億元
tensor(472, device='cuda:0') tensor(472, device='cuda:0')
model output ans 2 司
('昌公司負責人吳國昌被控和胞弟吳孝昌多名友人以加盟投資車開發觀光飯店名義超過24億元另有三名律師鑑證契約一審依違反銀行法判吳國昌14年徒刑二審改判8年徒刑最高法院撤銷發回更審高院更一審今天宣判判吳國昌12年徒刑沒收犯罪所得仍可上訴判決指出吳國昌將吸金業務分成業務飯店房地產三大部門由張欽堯等人擔任幹部指導旗下人員對外推銷投資產品抽取佣金以露營車售後租回專案及在嘉義金門買賣觀光景點周圍土地開發及興

('26萬元高級療程實做5萬元手術腫瘤名醫吳思遠涉詐遭約談台北市立萬芳醫院放射腫瘤科主任吳思遠醫師遭檢舉向病患推薦有放療神射手之稱的螺旋刀治療但實際手術卻改採較廉價的療程已知至少三名病患被騙資料照記者王善嬿攝2019-04-1306:00:00記者黃捷台北報導名醫遭爆詐欺病患台北市立萬芳醫院放射腫瘤科主任吳思遠醫師遭檢舉向病患推薦有放療神射手之稱的螺旋刀治療TomoTherapy但實際手術卻改採較廉價的療程已知至少三名病患被騙檢調懷疑吳與設備廠商承業生醫配合拆帳昨搜索約談吳與業者等六人全案朝詐欺罪方向偵辦疑與設備廠商配合拆帳吳思遠除身為萬芳醫院放腫科主任也具有健保審查委員台北醫學大學醫學院專任助理教授等身分在醫界頗負聲望承業生醫是國內第一個與醫療院所合建癌症放射治療設備的公司與十多間醫院合作腫瘤醫療中心穩坐國內放療設備龍頭寶座請繼續往下閱讀3名患者已有2人死亡檢調獲報吳思遠於二○一二年間向三名腫瘤患者推薦廿六萬元的螺旋刀自費療程實際放療時卻施作僅需自費五至六萬元的便宜療程三名患者中目前已有兩人死亡此外病患自費的放療費用診察費已含括其中不得再申請補助吳卻詐領健保費相關行為涉嫌詐欺台北地檢署昨',)
ans is:  吳思遠
tensor(370, device='cuda:0') tensor(491, device='cuda:0')
model output ans 2 下閱讀3名患者已有2人死亡檢調獲報吳思遠於二○一二年間向三名腫瘤患者推薦廿六萬元的螺旋刀自費療程實際放療時卻施作僅需自費五至六萬元的便宜療程三名患者中目前已有兩人死亡此外病患自費的放療費用診察費已含括其中不得再申請補助吳卻詐領健保費相關行為涉
tensor(370, device='cuda:0') tensor(497, device='cuda:0')
model output ans 2 下閱讀3名患者已有2人死亡檢調獲報吳思遠於二○一二年間向三名腫瘤患者推薦廿六萬元的螺旋刀自費療程實際放療時卻施作僅需自費五至六萬元的便宜療程三名患者中目前已有兩人死亡此外病患自費的放療費用診察費已含括其中不得再申請補助吳卻詐領健保費相關行為涉嫌詐欺台北地
tensor(447, device='cuda:0') tensor(491, device='cuda:0')
model output ans 2 

In [37]:
# testing topk 


BATCH_SIZE = 1
config = BertConfig.from_pretrained(my_pretrain,output_hidden_states=True)



class TestDataset(Dataset):
    def __init__(self, 
                 input_ids,
                 token_types, 
                 attention_mask,
                 start_pos_label, 
                 end_pos_label, text):
        self.input_ids = input_ids
        self.token_type_ids = token_types
        self.attention_mask = attention_mask
        self.start_pos_labels = start_pos_label
        self.end_pos_labels = end_pos_label
        self.text = text
    def __getitem__(self,idx):
        inputid = np.array(self.input_ids[idx])
        tokentype = np.array(self.token_type_ids[idx])
        attentionmask = np.array(self.attention_mask[idx])
        start_pos = self.start_pos_labels[idx]
        end_pos =  self.end_pos_labels[idx]
        text = self.text[idx]
        return inputid , tokentype , attentionmask,  start_pos , end_pos , text
    
    def __len__(self):
        return len(self.input_ids)

testset = TestDataset(test_input_ids,
                        test_token_types,
                        test_attention_mask,
                        test_start_pos_labels,
                        test_end_pos_labels,
                       test_x)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = BertModel.from_pretrained('./pretrain_roberta_on_TBdata/',config=config)
model = model.to(device)
model.output_hidden_states = True
model.eval()


pos_model = torch.load('./TB_multispan/pos_model_single_task_QAdata.pkl')
pos_model.eval()
pos_loss = nn.CrossEntropyLoss()


with torch.no_grad():
    for data in testloader:
        
        tokens_tensors ,  segments_tensors , masks_tensors, start_pos_labels , end_pos_labels = [t.to(device) for t in data[:5]]
        
        text = data[-1]
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        

        bert_all_768 = bert_outputs[0]
        
        mini_batch = bert_all_768.size()[0]
        optimizer.zero_grad()
        start_pred , end_pred = pos_model(bert_all_768,bert_all_768)
        
        start_pred = start_pred.reshape((mini_batch,512))
        end_pred = end_pred.reshape((mini_batch,512))


#         start_pred = nn.functional.log_softmax(start_pred,dim=1)
#         end_pred = nn.functional.log_softmax(end_pred,dim=1)
        
#         start_pos_labels = torch.softmax(start_pos_labels,dim=1)
#         end_pos_labels = torch.softmax(end_pos_labels,dim=1)
        
        start_pos_labels = torch.argmax(start_pos_labels,dim=1)
        end_pos_labels = torch.argmax(end_pos_labels,dim=1)
        
        
        if(start_pos_labels[0] > 0):
            print('===========================')
            print(text)
            print('ans is: ', text[0][0][start_pos_labels[0]-1:end_pos_labels[0]-1] , start_pos_labels[0]-1 , end_pos_labels[0]-1)
            print('===========================')
        else:
            continue
            
        topk = 3
        
        start_topk_indices = torch.topk(start_pred, topk).indices
        end_topk_indices = torch.topk(end_pred,topk).indices
        
        
        for i in range(topk):
            start_index = start_topk_indices[0][i]
            end_index = end_topk_indices[0][i]
            
            start_index -= 1
            end_index -= 1
            
            print('start_index: ' ,start_index)
            print('end_index: ', end_index)
#             if(end_index > start_index or abs(end_index-start_index)>20):
#                 continue
            try:
        
                if(start_index < end_index):
                    print('model output ans ' + str(i) + ' ' +  text[0][0][start_index:end_index])
                else:
                    print('model output ans ' + str(i) + ' ' +  text[0][0][end_index:start_index])
            except:
                print('out of range')
#             break

#         break
        
        
        




device: cuda:0
[('蘋果西打母公司掏空案總經理聲押禁見董座15萬交保大飲總經理孫幼英昨深夜移送新北檢複訊訊後遭聲押禁見記者陳慰慈攝2019-04-1206:00:12記者陳慰慈新北報導生產知名飲料蘋果西打的大西洋飲料公司1213因財報難產爆出掏空疑雲目前股票已暫停交易檢調昨依違反證券交易法非常規交易特別背信罪等罪約談大飲董事長江國貴登記負責人總經理孫幼英實際負責人獨立董事林偉強蘇芸樂財務經理鍾素娥張玉鳳等6人到案蘇芸樂昨晚訊後以3萬5千元交保今凌晨2時許依涉犯重罪有串證逃亡之虞將孫幼英聲押禁見江國貴15萬元林偉強10萬元鍾素娥40萬元張玉鳳25萬元交保大飲公司近日深陷財務經營風暴去年第3季與關係人簽約以64億元取得位於台南佳里高雄湖內及新北市新店等3筆不動產而與大飲交易的國信旭順公司董事名單高度重疊大飲總經理孫幼英身兼國信董事且國信又是大飲旭順的監察人旭順的監察人則是國信公司治理惹議請繼續往下閱讀此外國信旭順公司登記地址也是大飲位於新北市三重區的辦公室等於都是大飲能實質掌控的公司讓外界有左手賣右手的掏空質疑而林偉強蘇芸樂分別擔任國信食品代協理國信食品課長卻又是大飲獨立董事不符金管會規定讓會計師對大飲去年財',), ('洗錢',)]
ans is:  林偉強 tensor(177, device='cuda:0') tensor(180, device='cuda:0')
start_index:  tensor(-1, device='cuda:0')
end_index:  tensor(-1, device='cuda:0')
model output ans 0 
start_index:  tensor(487, device='cuda:0')
end_index:  tensor(487, device='cuda:0')
model output ans 1 
start_index:  tensor(484, device='cuda:0')
end_index:  tensor(484, device='cuda:0')
model output ans 2 
[('參選嘉義市東區議員失利的藍力凱及梅山鄉民代表當選人許文豪因競選期間涉嫌賄選嘉義地檢署檢察官22日依選罷法第99條第1項對2人提起公訴並針對許文豪提出當選無效之訴2人目前

[('金錢豹餐飲集團等擔任工程部副理的洪勝明負責寶麗金餐飲集團下的聚品軒婚宴會館寶麗金鑫悅富餐廳-崇德店裝修工程涉嫌向多家承攬廠商索取5至15回扣共收取回扣1318萬8000元經老闆袁昶平發現向調查局台中市調查處提出檢舉台中地檢署今天依背信罪將洪勝明起訴洪勝明否認背信犯行辯稱他是鼎盛冷氣行的技術師不清楚鼎盛冷氣行是否為告訴人首元公司的關係企業他沒有太大權力任何事情都是范姓工程處處長所決定他向廠商所收取的不是回扣而是工程介紹費且未曾對廠商施壓或刁難洪的律師也辯護稱得標施作的廠商僅是將利潤撥付一些給洪感謝洪介紹工作機會起訴書指出洪勝明52歲於2004年間開始受雇於袁昶平所經營的金錢豹餐飲機構於袁昶平在2015年設立首元投資開發股份有限公司後仍繼續在該公司旗下的藏格教育集團金錢豹娛樂集團及寶麗金餐飲集團等事業體內擔任工程部副理2016年2月間洪勝明經袁昶平指示與不知情的范姓工程處處長負責寶麗金集團下的聚品軒婚宴會館整修工程時以收取工程介紹費向承攬廠商收取工程款5至15的回扣損害首元公司的財產及利益同年間洪勝明向施作聚品軒婚宴會館木作裝修工程的慧友室友設計公司黃姓業者索取回扣106萬向施作天花板木工',), ('洗錢',)]
ans is:  洪勝明 tensor(16, device='cuda:0') tensor(19, device='cuda:0')
start_index:  tensor(16, device='cuda:0')
end_index:  tensor(484, device='cuda:0')
model output ans 0 洪勝明負責寶麗金餐飲集團下的聚品軒婚宴會館寶麗金鑫悅富餐廳-崇德店裝修工程涉嫌向多家承攬廠商索取5至15回扣共收取回扣1318萬8000元經老闆袁昶平發現向調查局台中市調查處提出檢舉台中地檢署今天依背信罪將洪勝明起訴洪勝明否認背信犯行辯稱他是鼎盛冷氣行的技術師不清楚鼎盛冷氣行是否為告訴人首元公司的關係企業他沒有太大權力任何事情都是范姓工程處處長所決定他向廠商所收取的不是回扣而是工程介紹費且未曾對廠商施壓或刁難洪的律師也辯護稱得標施作的廠商僅是將利潤撥付一些給洪感謝洪介紹工作機會起訴書指出洪勝明52歲於2004年間開始受雇於袁昶平所經營的金錢豹餐飲機構於袁昶平在2015年設立首元投資開發股份有限公司後仍繼續在該公司旗下

[('鄭銘富走私運毒集團將K他命藏在人造石板從大陸廈門港裝貨櫃運抵基隆港去年5月29日遭檢調查獲其中鄭姓等5人同步在大陸落網在台接頭的男子高振殷依運輸3級毒品起訴士林地院認定兩岸司法互助協議取得大陸地區公安訊問鄭等5人筆錄具有效力高受僱接應純質淨重575公斤K毒判處有期徒刑10年可上訴鄭銘富以每月2萬5000元的薪資雇用高在台接應毒品他自己則與周正華呂家緯李春生和黃建強在大陸將人造石板挖空分別將35包Ｋ毒塞在2塊石板內混雜在其他厚重石板當中從廈門港裝船隔天即運抵基隆港基隆檢調港務局等人利用X光發現其中異狀按兵不動直到運送到林口倉庫大隊人馬將高逮捕同時透過兩岸共同打擊犯罪的機制在東莞吉林等地逮捕鄭姓首腦目前5人仍在大陸尚未移交給我方在台負責接應的高男曾在大陸酒店擔任少爺除幫忙匯款給不知情的貨運公司6萬7000多元運費還簽收運毒的貨櫃檢察官取得鄭等人在公安訊問時的筆錄作為呈堂證供查扣的手機內有鄭高的通話紀錄證明高受僱於鄭等人合議庭認定近700公斤的K他命純質淨重575公斤數量龐大歷年罕見一旦流入市面將戕害國人健康高男所犯不值得同情★中時電子報關心您保護自己遠離毒品中時)',), ('洗錢',)]
ans is:  高振殷 tensor(66, device='cuda:0') tensor(69, device='cuda:0')
start_index:  tensor(0, device='cuda:0')
end_index:  tensor(474, device='cuda:0')
model output ans 0 鄭銘富走私運毒集團將K他命藏在人造石板從大陸廈門港裝貨櫃運抵基隆港去年5月29日遭檢調查獲其中鄭姓等5人同步在大陸落網在台接頭的男子高振殷依運輸3級毒品起訴士林地院認定兩岸司法互助協議取得大陸地區公安訊問鄭等5人筆錄具有效力高受僱接應純質淨重575公斤K毒判處有期徒刑10年可上訴鄭銘富以每月2萬5000元的薪資雇用高在台接應毒品他自己則與周正華呂家緯李春生和黃建強在大陸將人造石板挖空分別將35包Ｋ毒塞在2塊石板內混雜在其他厚重石板當中從廈門港裝船隔天即運抵基隆港基隆檢調港務局等人利用X光發現其中異狀按兵不動直到運送到林口倉庫大隊人馬將高逮捕同時透過兩岸共同打擊犯罪的機制在東莞吉林等地逮捕鄭姓首腦目前5人仍在大陸尚未移交給我方在台負責接應的高男曾

[('前網虎公司創辦人跨越科技董事長李奇申為取得陸客來台通關查驗系統兩項標案涉嫌以餽贈股票現金投資乾股代繳房貸等四大行賄手法買通內政部資訊中心主任移民署前資訊組長施明德護航得標台北地檢署昨天兵分廿一路搜索施李等人住居所帶回十三名官商人頭漏夜偵訊全案依違背職務行收賄洗錢罪偵辦除李奇申施明德被偵訊外廉政署也同步約談施明德妻子翁美惠姊姊張施麗滿姊夫張慶宗大嫂施吳素貞岳母賴月菊李奇申女友黃詩涵到案內政部表示部長徐國勇上任後發覺施的行為有異已免去他管理採購業務權限及擔任各項評委資格並向廉政署表達勿枉勿縱立場施明德曾經任行政院研考會高級分析師榮獲模範移民官他涉嫌在擔任移民署資訊組長期間利用經手入出境資訊系統招標案機會收取多家廠商賄賂再洩漏標案資訊供廠商圍綁標至少有七件標案涉弊北檢前年會同廉政署動員上百人大搜索約談四十二名官商當時施明德與李奇申各被檢方交保一百萬元據悉檢廉兩年前搜索黃詩涵住處時查扣一本跨越科技股東名冊發現裡面記載張慶宗名字旁邊卻用括弧註記施明德字樣北檢認為內情不單純歷經兩年鴨子划水暗中蒐證確認施明德透過妻子姊夫名義收受李奇申餽贈四百張股票協助跨越科技取得移民署二○○九年與二○一○年發包的外',), ('洗錢',)]
ans is:  施吳素貞 tensor(174, device='cuda:0') tensor(178, device='cuda:0')
start_index:  tensor(15, device='cuda:0')
end_index:  tensor(101, device='cuda:0')
model output ans 0 李奇申為取得陸客來台通關查驗系統兩項標案涉嫌以餽贈股票現金投資乾股代繳房貸等四大行賄手法買通內政部資訊中心主任移民署前資訊組長施明德護航得標台北地檢署昨天兵分廿一路搜索施李
start_index:  tensor(78, device='cuda:0')
end_index:  tensor(-1, device='cuda:0')
model output ans 1 
start_index:  tensor(-1, device='cuda:0')
end_index:  tensor(499, device='cuda:0')
model output ans 2 
[('成立60年的南港輪胎公司是台

[('法尼新創科技號稱有全台最大以太幣礦場負責人田書旗與妻子黃淑霞打出挖礦機月利率百分三吸引投資客涉嫌違法六億元今年二月資金空缺爆發財務危機投資客拿不到利息驚覺受害田黃因違反銀行法遭羈押幾乎每天都有人到地檢署提告據了解田書旗過去擔任控股公司講師負責到各地分公司講解投資方案後來擔任百易新創集團總裁旗下的京稜旅行社也承辦包機直飛越南芽莊行程今年四月傳出疑因資金問題導致班機停飛三百多名旅客受影響檢調查出田書旗四十一歲和妻子黃淑霞四十四歲二○一六年創立法尼總部設在桃園龍潭工業區一處透天厝以挖礦投資以太幣分別用十五萬廿六萬廿七萬五千元等三種方式向投資客出售挖礦機號稱每月可獲百分之三並簽約為期一年若不想續約還會歸還本金不少人剛開始每月都能獲利法尼也打出每拉一個下線就可獲三萬五千元紅利而為取信投資客還會帶大家參觀公司相關企業的實體店面包括火鍋店咖啡廳飲料店及二手車招攬過程也開立統一發票聲稱其正當合法性有被害人說法尼去年底聲稱礦機限量要買要快鼓吹大家瘋狂加碼未料加速吸金後沒多久就因後金補不齊前金傳出財務空洞今年二月發不出利息導致許多人砸大錢卻血本無歸據了解桃園地檢署去年底接獲情資蒐證數月後今年四月收網田黃到',), ('洗錢',)]
ans is:  黃淑霞 tensor(27, device='cuda:0') tensor(30, device='cuda:0')
start_index:  tensor(-1, device='cuda:0')
end_index:  tensor(-1, device='cuda:0')
model output ans 0 
start_index:  tensor(500, device='cuda:0')
end_index:  tensor(503, device='cuda:0')
model output ans 1 
start_index:  tensor(503, device='cuda:0')
end_index:  tensor(500, device='cuda:0')
model output ans 2 
[('記者黃捷台北報導毒販林輝宏1993年涉嫌以免費旅遊泰國為餌招攬6名男子吞食海洛因毒球方式從泰國走私毒品入台是國內首起吞毒球走私案後有2人因毒球在體內破裂毒發身亡林男事後遭控走私運毒及殺人罪但他逃亡已久運

...

[('前黑澀會美眉薔薔林嘉凌再爆出吸毒疑雲檢警接獲線報指稱北市大直一處豪宅有人辦毒趴台北地檢署檢察官11日晚間指揮北市警內湖分局前往大直豪宅調查當場發現薔薔林嘉凌與男女友人共12人現場並起獲大麻K他命等毒品據了解林嘉凌等人否認吸毒警方將12人依毒品罪送辦檢方訊後請回林嘉凌另11人分獲3萬元1萬元交保限制住居請回這已經是薔薔林嘉凌第4次因毒品被逮2015年她與3名友人到新北市某摩鐵開趴遭警方臨檢查獲薔薔林嘉凌是前電視節目我愛黑澀會成員因外形豔麗身材火辣常在我愛黑澀會節目中扮醜搞笑被戲稱諧星薔但她作風備受爭議頻爆偷竊吸毒販毒等負面新聞雖均未被起訴卻傷及個人形象林嘉凌父親林茂樹因遭中華地政士事務所涉嫌騙屋詐貸檢警逮捕主嫌陳國帥林嘉凌還曾同陪父親到士林地檢署召開說明會檢警日前接獲線報指北市大直一處豪宅有人辦毒趴台北地檢署檢察官11日晚間指揮北市內湖警分局前往大直豪宅調查當場發現薔薔林嘉凌與男女友人共12人現場並起獲大麻K他命等毒品據了解林嘉凌等人否認吸毒對於毒品來源互推責任警方將12人依毒品罪送辦檢方訊後請回林嘉凌另宋姓人士交保3萬陳姓楊姓人士2人各交保1萬紀姓廖姓人士2人限制住居其餘人士請回★中時',), ('洗錢',)]
ans is:  陳國帥 tensor(308, device='cuda:0') tensor(311, device='cuda:0')
start_index:  tensor(8, device='cuda:0')
end_index:  tensor(11, device='cuda:0')
model output ans 0 林嘉凌
start_index:  tensor(74, device='cuda:0')
end_index:  tensor(279, device='cuda:0')
model output ans 1 薔林嘉凌與男女友人共12人現場並起獲大麻K他命等毒品據了解林嘉凌等人否認吸毒警方將12人依毒品罪送辦檢方訊後請回林嘉凌另11人分獲3萬元1萬元交保限制住居請回這已經是薔薔林嘉凌第4次因毒品被逮2015年她與3名友人到新北市某摩鐵開趴遭警方臨檢查獲薔薔林嘉凌是前電視節目我愛黑澀會成員因外形豔麗身材火辣常在我愛黑澀會節目中扮醜搞笑被戲稱諧星薔但她作風備受爭議頻爆偷竊吸毒販毒等負面新聞雖均未被起訴卻傷及個人形象林


[('勝威國際集團未經主管機關許可設立多福多財VV農場遊戲等方案3年來向民眾吸金34億元涉嫌違反銀行法及多層次傳銷管理法台中地檢署今天上午聲押主嫌李承孺及核心幹部詹國霆黃清政等3人法官裁定李等3人羈押禁見至於集團的幹部徐沛清陳享恩陳聰明林義傑蔡金澄陳宜鑫等5人分別以10萬至30萬元不等交保主嫌李承孺核心幹部詹國霆黃清政等人到案後坦承的確有這樣的計劃投資有獎金制度但他們不是經營決策者只是分享投資檢察官不採信向法官聲押3人獲准檢調指出勝威國際集團自105年9月起在台設立據點負責人是李承孺黃男詹男陳男等人是核心幹部設立多福多財5個投資方案由民眾投資美金100500100050001萬元不等取得顯不相當的高額報酬至107年6月起轉變為VV農場遊戲投資1年取得顯不相當的高額報酬經法務部調查局基隆調查站報請台中地檢署偵辦經多時蒐證後認定勝威集團在台負責人李承儒涉嫌違法銀行法重大檢察官昨天指揮上百調查員兵分20路前往台北市基隆市桃園市台中市等地區傳訊被告及證人共22人到案帶到法務部調查局中機站漏夜偵訊檢察官訊後認定李承孺詹國霆黃清政等3人涉嫌違反銀行法多層次傳銷管理法重大今天上午聲押李等3人獲准另幹部徐沛',), ('洗錢',)]
ans is:  林義傑 tensor(115, device='cuda:0') tensor(118, device='cuda:0')
start_index:  tensor(80, device='cuda:0')
end_index:  tensor(72, device='cuda:0')
model output ans 0 孺及核心幹部詹國
start_index:  tensor(69, device='cuda:0')
end_index:  tensor(-1, device='cuda:0')
model output ans 1 
start_index:  tensor(105, device='cuda:0')
end_index:  tensor(481, device='cuda:0')
model output ans 2 部徐沛清陳享恩陳聰明林義傑蔡金澄陳宜鑫等5人分別以10萬至30萬元不等交保主嫌李承孺核心幹部詹國霆黃清政等人到案後坦承的確有這樣的計劃投資有獎金制度但他們不是經營決策者只是分享投資檢察官不採信向

[('超跑達人坑富二代勾結竹聯前堂主詐上億超跑達人鄧超鴻左圖涉嫌吸金臉書常曬酷炫跑車照片右圖記者邱俊福翻攝自臉書2019-07-2505:30:00投資黃金失利淪詐騙記者邱俊福台北報導超跑車壇頗具知名度的BetaAuto北塔車業老闆超跑達人鄧超鴻去年間因投資黃金期貨失利資金出現缺口後以誆稱買賣超跑與黃金等投資標的向許多富二代吸金詐騙藉此解決公司財務危機且只要被害人出面索討便由前竹聯幫玄武堂堂主背景的公司股東道克明出面恫嚇刑事局日前將鄧道與公司2員工共4人查緝到案初估吸金詐欺上億元黑幫股東恫嚇被害人警方初步調查38歲鄧男積極擴展超跑市場為打響知名度常舉辦車聚公司全盛時期常停有10多輛超跑每月有數百萬元獲利但去年間鄧男投資黃金期貨損失上億元財務出現危機只好將公司的超跑賣給租賃公司再付租金租回公司同時也向富二代等金主吸金行騙最後公司仍因債務沉重倒閉請繼續往下閱讀鄧男到案後稱要給股東分紅還債才出此下策道男則喊冤表示沒有參與行騙自己也是投資公司的被害人還借錢給鄧男而出面要求被害人不要上門討債只是好心幫忙希望被害人能讓鄧男賺錢後再慢慢歸還沒有恐嚇警方表示鄧男鼓吹被害人以買賣超跑或黃金空運到國外交易賺取價',), ('洗錢',)]
ans is:  道克明 tensor(202, device='cuda:0') tensor(205, device='cuda:0')
start_index:  tensor(22, device='cuda:0')
end_index:  tensor(-1, device='cuda:0')
model output ans 0 
start_index:  tensor(-1, device='cuda:0')
end_index:  tensor(485, device='cuda:0')
model output ans 1 
start_index:  tensor(485, device='cuda:0')
end_index:  tensor(488, device='cuda:0')
model output ans 2 超跑或
[('差等投資且為取信被害人將租賃的高價跑車提供被害人作為擔保品惟為能持續詐騙其他被害人便向原被害人假借驗車保養客戶看車等名義將原作為擔保品的高價跑車騙回等事後被害人發覺有異到公司討債道男則出面以黑道

[('利誘街友當人頭1張芭樂票跳票98億男子林秉勳及盧文凱涉嫌販售芭樂票近3年跳票81億餘元資料照2019-08-1605:30:00記者陳慰慈新北報導男子林秉勳原名林文良及盧文凱涉嫌販售芭樂票近3年跳票81億餘元其中盧嫌一張支票跳票金額高達98億元致我國跳票金額暴增影響台灣在亞太防制洗錢組織APG洗錢防制評鑑結果新北地檢署昨依詐欺取財等罪嫌起訴林男盧男及共犯等7人林秉勳以街友當人頭開立公司販售俗稱芭樂票的空頭支票跳票4750萬餘元2012年被判刑2年4個月確定林男出獄後又重操舊業請繼續往下閱讀3年跳票81億7嫌起訴起訴指出2015年起林男以自己名義或情商不知情子女擔任公司負責人另以1萬5千元至2萬元不等代價利誘何姓馬姓謝姓譚姓街友與失業者擔任資本額最高50萬元的公司負責人並向銀行申請支票簿以每張支票2800元賣給芭樂票集團盤商並命楊男前往交付芭樂票林男則預知芭樂票集團成員特定的退票時間並依客戶需求開立不同票期長短的芭樂票2016年7月至去年4月林男共賣出3172張芭樂票獲利888萬餘元跳票50億餘元而男子盧文凱也透過仲介向林男找來當公司登記負責人的何姓街友購買15張芭樂票並開出5張超過億元',), ('洗錢',)]
ans is:  林秉勳 tensor(19, device='cuda:0') tensor(22, device='cuda:0')
start_index:  tensor(18, device='cuda:0')
end_index:  tensor(68, device='cuda:0')
model output ans 0 子林秉勳及盧文凱涉嫌販售芭樂票近3年跳票81億餘元資料照2019-08-1605:30:00記者陳慰
start_index:  tensor(65, device='cuda:0')
end_index:  tensor(25, device='cuda:0')
model output ans 1 凱涉嫌販售芭樂票近3年跳票81億餘元資料照2019-08-1605:30:00記
start_index:  tensor(70, device='cuda:0')
end_index:  tensor(468, device='cuda:0')
model output ans 2 北報導男子林秉勳原名林文良及盧文凱涉

[('記者劉昌松台北報導中華壘球協會傳貪污弊案2017年受國訓中心委託辦理女壘國手赴美培訓被檢調查出協會涉嫌勾結代辦的運佳旅行社浮報70多萬元的旅費而運佳的實際負責人還是中華奧會副主席蔡賜爵檢調10日發動搜索約談後台北地檢署11日凌晨依貪污等罪嫌諭令蔡20萬元交保壘球協會副秘書長畢鈞輝等6人以30萬到5萬不等交保▲中華奧會副主席蔡賜爵11日被檢察官依貪污罪嫌諭令20萬元交保圖記者劉昌松攝我國女壘隊在國際賽事中屢獲佳績為了爭取2020即將在東京展開的奧運賽事入場券政府早就開始積極培訓備戰國家運動訓練中心在2018年間委託中華民國壘球協會辦理的培訓專案就是希望藉由壘球協會的海外關係協助安排好手赴美培訓整個標案金額300多萬元採最有利標最後由運佳旅行社取得▲中華慢壘協會副秘書長畢鈞輝訊後也被檢察官諭令交保30萬元圖記者劉昌松攝但是檢調查出壘球協會疑似勾結運佳在旅費部分浮報了70多萬元雖然壘球協會屬民間單位但接受政府委託辦理培訓承辦人員就等於是受委託從事公務須受貪污治罪條例規範台北地檢署10日指揮高雄市調處兵分4路發動搜索約談壘球協會副秘書長畢鈞輝及運佳旅行社實際負責人中華奧會副主席蔡賜爵等9人據了',), ('洗錢',)]
ans is:  蔡賜爵 tensor(89, device='cuda:0') tensor(92, device='cuda:0')
start_index:  tensor(479, device='cuda:0')
end_index:  tensor(479, device='cuda:0')
model output ans 0 
start_index:  tensor(-1, device='cuda:0')
end_index:  tensor(-1, device='cuda:0')
model output ans 1 
start_index:  tensor(482, device='cuda:0')
end_index:  tensor(482, device='cuda:0')
model output ans 2 
[('桃園桃機工程弊案工程師認罪40萬交保2019-10-0905:30:00記者陳恩惠桃園報導桃園機場二期航廈擴建等多項工程八月間爆發桃園機場公司工程處官員涉嫌收受巨額回扣弊案桃園地檢署依貪污治罪條例聲押工

In [ ]:
BATCH_SIZE = 32
config = BertConfig.from_pretrained(my_pretrain,output_hidden_states=True)

trainset = TrainDataset(train_input_ids,
                        train_token_types,
                        train_attention_mask,
                        train_start_pos_labels,
                        train_end_pos_labels)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print("device:", device)
model = BertModel.from_pretrained('./pretrain_roberta_on_TBdata/',config=config)
model = model.to(device)
model.output_hidden_states = True
model.eval()

all_768_x = np.array([])
flag = True
with torch.no_grad():
    for data in trainloader:
        
        tokens_tensors ,  segments_tensors , masks_tensors = [t.to(device) for t in data][:3]
        
        
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        
        
        

        
        bert_all_768 = bert_outputs[0]
        tmp_768 = bert_all_768.clone().to('cpu').numpy()
        
        if flag:
            flag = False
            all_768_x = tmp_768
        else:
            all_768_x = np.concatenate((all_768_x,tmp_768),axis = 0)
            
print(all_768_x.shape)

        
with open('./TB_multispan/train_all_768_x.pickle', 'wb') as handle:
    pickle.dump(all_768_x, handle, protocol=pickle.HIGHEST_PROTOCOL) 


device: cuda:0


In [ ]:
print(one_count/95416)

In [67]:
with open('./TB_multispan/train_input_ids.pickle', 'wb') as handle:
    pickle.dump(train_input_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
with open('./TB_multispan/train_token_types.pickle', 'wb') as handle:
    pickle.dump(train_token_types, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./TB_multispan/train_attention_mask.pickle', 'wb') as handle:
    pickle.dump(train_attention_mask, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./TB_multispan/train_start_pos_labels.pickle', 'wb') as handle:
    pickle.dump(train_start_pos_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./TB_multispan/train_end_pos_labels.pickle', 'wb') as handle:
    pickle.dump(train_end_pos_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./TB_multispan/train_x.pickle', 'wb') as handle:
    pickle.dump(train_x, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
def clean_string(content):
    content = content.replace('\n','').replace('\t','').replace(' ','').replace('\xa0','')
    content = re.sub("[●▼►★]", "",content)
    return content

def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para) 
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    return para.split("\n")


def combine_sentence(sentences):
    li = []
    string = ''
    for k in range(len(sentences)):
        sentence = sentences[k]
        if len(string) + len(sentence) < 510:
            string = string + sentence
        else:
#             原本是空的代表sentences太常
            if string == '':
                n = 510
                tmp_li = [sentence[i:i+n] for i in range(0, len(sentence), n)]
                string = tmp_li.pop(-1)
                li = li + tmp_li
            else:
                li.append(string)
                string = sentence
    if(string != ''):
        li.append(string)
    return li



In [5]:
train_content = train_df['full_content'].tolist()
test_content = test_df['full_content'].tolist()
all_content = train_content + test_content
print(len(all_content))

4952


In [7]:
pretrain_content = []

for content in all_content:
    content = clean_string(content)
    cut_sentence = cut_sent(content)
    pretrain_content = pretrain_content + combine_sentence(cut_sentence)
print(len(pretrain_content))   
        

12528


In [8]:

pretrain_model_name = "hfl/rbtl3"
tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)
model = BertForPreTraining.from_pretrained(pretrain_model_name)

Some weights of BertForPreTraining were not initialized from the model checkpoint at hfl/rbtl3 and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
class PretrainDataset(Dataset):
    def __init__(self, input_dict):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        return inputid , tokentype , attentionmask
    
    def __len__(self):
        return len(self.input_ids)

In [10]:
input_dict = tokenizer.batch_encode_plus(pretrain_content, 
                                         add_special_tokens=True,
                                         max_length=512,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [11]:
BATCH_SIZE = 4
pretrainset = PretrainDataset(input_dict)
pretrainloader = DataLoader(pretrainset, batch_size=BATCH_SIZE)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
EPOCHS = 2
for epoch in range(EPOCHS):
    for data in pretrainloader:   
        tokens_tensors, segments_tensors, masks_tensors = [t.to(device) for t in data]
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, )
    model.save_pretrained('./rbtl3_pretrain')
    print('epoch:', epoch+1)

device: cuda:0
epoch: 1


In [ ]:
with open('./TB/contents_names.pickle', 'rb') as handle:
    contents_names = pickle.load(handle)